In [7]:
import os

In [8]:
%pwd

'c:\\Users\\abhay\\Desktop\\Internship\\Data-Engineering-Internship-Bosch'

In [3]:
os.chdir("../")

In [9]:
%pwd

'c:\\Users\\abhay\\Desktop\\Internship\\Data-Engineering-Internship-Bosch'

In [10]:
import pandas as pd

In [11]:
data = pd.read_csv("artifacts/data_ingestion/AirQualityUCI.csv")
data.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10-03-2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10-03-2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10-03-2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10-03-2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10-03-2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   float64
 3   PT08.S1(CO)    9357 non-null   float64
 4   NMHC(GT)       9357 non-null   float64
 5   C6H6(GT)       9357 non-null   float64
 6   PT08.S2(NMHC)  9357 non-null   float64
 7   NOx(GT)        9357 non-null   float64
 8   PT08.S3(NOx)   9357 non-null   float64
 9   NO2(GT)        9357 non-null   float64
 10  PT08.S4(NO2)   9357 non-null   float64
 11  PT08.S5(O3)    9357 non-null   float64
 12  T              9357 non-null   float64
 13  RH             9357 non-null   float64
 14  AH             9357 non-null   float64
 15  Unnamed: 15    0 non-null      float64
 16  Unnamed: 16    0 non-null      float64
dtypes: float64(15), object(2)
memory usage: 1.2+ MB


In [13]:
data.columns

Index(['Date', 'Time', 'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
       'PT08.S5(O3)', 'T', 'RH', 'AH', 'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')

In [14]:
from dataclasses import dataclass
from pathlib import Path

In [15]:
@dataclass(frozen = True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict
    failed_data_dir: Path
    valid_data_dir: Path


In [19]:
from src.dataEngineer.utils.common import read_yaml, create_directories
CONFIG_FILE_PATH = Path("config/config.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

In [17]:
class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir, config.failed_data_dir, config.valid_data_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema,
            failed_data_dir = config.failed_data_dir,
            valid_data_dir = config.valid_data_dir
        )

        return data_validation_config

In [18]:
import os
from src.dataEngineer import logger

In [ ]:
import shutil

from dataEngineer.entity.config_entity import DataValidationConfig


class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            data = data[[c for c in data.columns if not str(c).startswith("Unnamed")]]
            

            expected_dtype = self.config.all_schema
            data_dtype = dict(data.dtypes)

            for col, exp in data_dtype.items():
                if col not in expected_dtype.keys():
                    validation_status = False
                    shutil.move(self.config.unzip_data_dir, self.config.failed_data_dir)
                    logger.info(f"Extra column found in dataset at {self.config.unzip_data_dir} and moved to {self.config.failed_data_dir}")
                    break
                data_type = expected_dtype[col]  
                if  str(data_type) != str(exp):
                    validation_status = False
                    shutil.move(self.config.unzip_data_dir, self.config.failed_data_dir)
                    logger.info(f"Extra column found in dataset at {self.config.unzip_data_dir} and moved to {self.config.failed_data_dir}")
                    break
                    
                else:
                    validation_status = True
            if validation_status == True:
                shutil.move(self.config.unzip_data_dir, self.config.valid_data_dir)
                logger.info(f"Data is Valid, moved to {self.config.valid_data_dir} and ready to use for further processing")
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")
                f.close()


            return validation_status
        except Exception as e:
            raise e        
            

In [15]:

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-08-16 12:34:12,613: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-16 12:34:12,618: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-16 12:34:12,622: INFO: common: created directory at : artifacts]
[2025-08-16 12:34:12,625: INFO: common: created directory at : artifacts/data_validation]
[2025-08-16 12:34:12,627: INFO: common: created directory at : artifacts/quarantine]
[2025-08-16 12:34:12,628: INFO: common: created directory at : artifacts/valid_data]
[2025-08-16 12:34:12,652: INFO: 1035479745: Data is Valid, moved to artifacts/valid_data and ready to use for further processing]
